## Propertime Quickstart

Propertime is an attempt at implementing proper time management in Python, by fully embracing the extra complications arising due to the intrinsic need of conflating together physical and calendar time instead of neglecting them.

In a nutshell, it provides two main classes: the ``Time`` class for representing time (similar to a datetime) and the ``TimeSpan`` class for representing spans of time (similar to a timedelta). Such classes play nice with Python datetimes so that you can mix and match and use them only when needed.

You can have a look at the [README](https://github.com/sarusso/Propertime/blob/main/README.md) for a better introduction, some example usage and more info about Propertime.

This is the quickstart notebook. You might also want to check out the [API documentation](https://propertime.readthedocs.io).

### The Time class

The Time class is how Propertime represents time.

In [1]:
from propertime import Time

To get the time right now, just create a new Time instance without any arguments:

In [2]:
Time()

Time: 1708700327.0 (2024-02-23 14:58:47 UTC)

The main concept in Propertime is that time is always expressed as Epoch seconds (from 1st January 1970 UTC). Then, it can be represented in different ways or on different time zones and with different offsets. But at its core, a Time objects is a floating point number, and can always be casted as such:

In [3]:
float(Time())

1708700327.0

To get the current time on another timezone or using an offset, use the `tz` of `offset` arguments:

In [4]:
Time(tz='Europe/Rome')

Time: 1708700327.0 (2024-02-23 15:58:47 Europe/Rome)

In [5]:
Time(tz=None, offset=-72000)

Time: 1708700327.0 (2024-02-22 18:58:47 -20:00)

To instead create a Time instance at a given time, either use Epoch seconds, classic datetime-like arguments.

In [6]:
Time(1703520720.0)

Time: 1703520720.0 (2023-12-25 16:12:00 UTC)

In [7]:
Time(2023,12,3,16,12,0)

Time: 1701619920.0 (2023-12-03 16:12:00 UTC)

You can also instantiate Time classes form (ISO) string representations or a datetime objects, using their respective class methods. Using naive ISO strings or datetimes without, explicitly setting the time zone or offset, is not allowed (Propertime does not allow in general for naive time).

In [8]:
Time.from_iso('2023-12-25T16:12:00+01:00', tz='Europe/Rome')

Time: 1703517120.0 (2023-12-25 16:12:00 Europe/Rome)

In [9]:
from datetime import datetime
Time.from_dt(datetime(2023,12,3,16,12,0), tz='America/Los_Angeles')

Time: 1701648720.0 (2023-12-03 16:12:00 America/Los_Angeles)

..and you can always convert a Time object to its (ISO) string representation or a datetime:

In [10]:
Time(2023,12,3,16,12,0).to_iso()

'2023-12-03T16:12:00+00:00'

In [11]:
Time(2023,12,3,16,12,0).to_dt()

datetime.datetime(2023, 12, 3, 16, 12, tzinfo=<UTC>)

Once a Time object has been initialized, it behaves exactly as another numeric type (as int and float):

In [12]:
Time(1970,1,30,0,0,0) / 2

Time: 1252800.0 (1970-01-15 12:00:00 UTC)

If some operations are not supported by the Time class but are implemented for the float type, they will also work, making it possible to just plug-in Time objects in nearly any numerical utility, which is very useful in some circumstances when performing data analysis. An example with the square root:

In [13]:
import math
math.sqrt(Time(2023,12,3,16,12,0))

41250.69599412839

Given that Time objects behave as numerical types, many common operations are very intuitive. For example, given a set of arrival times, to compute the average arrival time you just need to do:

In [14]:
arrival_times = [Time(2023,12,3,16,12,0),
                 Time(2023,12,3,16,56,0),
                 Time(2023,12,3,16,31,0)]
sum(arrival_times)/len(arrival_times)

Time: 1701621180.0 (2023-12-03 16:33:00 UTC)

To instead compute the time elapsed (in seconds) between the first and the last arrival:

In [15]:
float(max(arrival_times) - min(arrival_times))

2640.0

..which in this case has been converted to float as the resulting value would not mean much in terms of "calendar" time.

With this definition of time, a difference between two time points is indeed just another number on the time axis (in seconds), and not a "delta" object as the timedelta for datatimes. To represent "deltas" of time, Propertime makes indeed use of a slightly different concept: the *time span*.

### The TimeSpan class
Time spans impement precise, calendar-awarare time arithmetic. Their main characteristic is to embarace that, as soon a calendar time component kicks-in, a time span can have *variable* duration: a day can indeed last either 23, 24 or 25 hours, while an hour will instead alwyas last 3600 seconds (leap seconds apart).

Morover, time spans' arithmetic fully acknowledge that some time manipulation operations are not always well defined (e.g. adding a month to the 31st of January) and that should raise an error, exactly as a divison by zero would.

Time spans can be instantiated either by by manually setting all of their components (years, months, weeks, days, hours, minutes, seconds and microseconds), or using their string representation: `1s` for one second, `1m` for one minute, `1h` for one hour, `1D` for one day, `1M` for one month, `1W` for one weelk and `1Y` for one year. Values other than "one" are of course supported, as well as combinations (to a certian degree).

For example:

In [16]:
from propertime import TimeSpan
TimeSpan('1h')

1h

More in detail, the following two lines contian equivalent instantiations:

In [17]:
assert TimeSpan('1h') == TimeSpan('3600s') == TimeSpan(seconds=3600) == TimeSpan(hours=1)
assert TimeSpan('1h_30m') == TimeSpan(hours=1, minutes=30)

Time spans can always be obtained as seconds with the ``as_seconds()`` method, which for fixed-duration time spans is always well defined:

In [18]:
TimeSpan('1h_30m').as_seconds()

5400.0

...while for variable-duration time spans (involving at least one calendar component) it is not:

In [19]:
try:
    TimeSpan('1D').as_seconds()
except Exception as e:
    print(e)

You can ask to get a calendar TimeSpan as seconds only if you provide the span starting point


This is because, as already mentioned, the duration of a time span involving calendar components it is not defined unless contextualised at a given time and on a given time zone: how many seconds a one-day time span lasts it dependends on the day. The `starting_at` argument servers for this puropose:

In [20]:
TimeSpan('1D').as_seconds(starting_at=Time(2023, 3, 26, 0, 0, 0, tz='Europe/Rome'))

82800.0

Propertime tries to handle such behaviour as clearly as possible, and for example it allows to explicitly state that a (fixed) 24-hours time span is different than a one-day time span:

In [21]:
assert TimeSpan('24h') == TimeSpan('86400s') 
assert TimeSpan('1D') != TimeSpan('86400s')

Time spans can be added and subtracted each other and to Time and datetime objects, in which case their context is automatically defined, thus allowing to maniplate time in an easy and consistent way:

In [22]:
Time(2023,1,1,0,0,0) + TimeSpan('1M')

Time: 1675209600.0 (2023-02-01 00:00:00 UTC)

Time spans can also be used to round, ceil or floor time.

In [23]:
TimeSpan('1D').round(Time(2023,10,29,16,0,0))

Time: 1698624000.0 (2023-10-30 00:00:00 UTC)

In [24]:
TimeSpan('1D').floor(Time(2023,1,31,19,21,47))

Time: 1675123200.0 (2023-01-31 00:00:00 UTC)

In [25]:
TimeSpan('1D').shift(Time(2023,1,31,19,21,47))

Time: 1675279307.0 (2023-02-01 19:21:47 UTC)

This is very useful for "traveling" around. For example, to get to the noon of the next day:

In [26]:
time = Time(2023,10,29,0,15,39, tz='Europe/Rome')
TimeSpan('1D').ceil(time) + TimeSpan('12h')

Time: 1698663600.0 (2023-10-30 12:00:00 Europe/Rome)

Time spans are also useful for "slotting" time, while taking care about all the extra DST complications. For example, to slot a day in 1-hour bins: 

In [27]:
start = Time(2023,10,29,0,0,0, tz='Europe/Rome')
end = start + TimeSpan('1D')

slot_strart_time = start
while slot_strart_time < end:
    print(slot_strart_time)
    slot_strart_time = slot_strart_time + TimeSpan('1h')

Time: 1698530400.0 (2023-10-29 00:00:00 Europe/Rome DST)
Time: 1698534000.0 (2023-10-29 01:00:00 Europe/Rome DST)
Time: 1698537600.0 (2023-10-29 02:00:00 Europe/Rome DST)
Time: 1698541200.0 (2023-10-29 02:00:00 Europe/Rome)
Time: 1698544800.0 (2023-10-29 03:00:00 Europe/Rome)
Time: 1698548400.0 (2023-10-29 04:00:00 Europe/Rome)
Time: 1698552000.0 (2023-10-29 05:00:00 Europe/Rome)
Time: 1698555600.0 (2023-10-29 06:00:00 Europe/Rome)
Time: 1698559200.0 (2023-10-29 07:00:00 Europe/Rome)
Time: 1698562800.0 (2023-10-29 08:00:00 Europe/Rome)
Time: 1698566400.0 (2023-10-29 09:00:00 Europe/Rome)
Time: 1698570000.0 (2023-10-29 10:00:00 Europe/Rome)
Time: 1698573600.0 (2023-10-29 11:00:00 Europe/Rome)
Time: 1698577200.0 (2023-10-29 12:00:00 Europe/Rome)
Time: 1698580800.0 (2023-10-29 13:00:00 Europe/Rome)
Time: 1698584400.0 (2023-10-29 14:00:00 Europe/Rome)
Time: 1698588000.0 (2023-10-29 15:00:00 Europe/Rome)
Time: 1698591600.0 (2023-10-29 16:00:00 Europe/Rome)
Time: 1698595200.0 (2023-10-29 17:

Lastly, note that the two the examples above involved a DST cange, which was correcly handled in both cases.

### Consistency and edge cases

One of the main features of Propertime is to enforce consistency at differente levels and to handle quite well several edge cases that are usually not handled by Python built-in time management utilities nor other third-party libraries.

Let's for example see what happens if we try to add a month to the 31st of january, which is an operation not well defined:

In [28]:
try:
    Time(2023,1,31,0,0,0) + TimeSpan('1M')
except Exception as e:
    print(e)

Day is out of range for month for 2023-01-31 00:00:00+00:00 plus 1 month(s)


Similarly, let's see if we add one day to the 2:15 AM *before* a DST change when skipping an hour:

In [29]:
try:
    Time(2023,3,25,2,15,0, tz='Europe/Rome') + TimeSpan('1D')
except Exception as e:
    print(e)

Cannot shift "2023-03-25 02:15:00+01:00" by "1D" (Sorry, time 2023-03-26 02:15:00 does not exist on time zone Europe/Rome)


This happens because 2:15 AM on March 26th, 2023 actually does not exists on the `Europe/Rome` time zone. And neither 2:15 AM on March 12th, 2023 on `America/New_York` does:

In [30]:
try:
    Time(2023,3,12,2,15,0, tz='America/New_York')
except Exception as e:
    print(e)

Sorry, time 2023-03-12 02:15:00 does not exist on time zone America/New_York


A similar logic applies when the DST adjustment goes in the other way, by duplicating an hour:

In [31]:
try:
    Time(2023,10,28,2,15,0, tz='Europe/Rome')+ TimeSpan('1D')
except Exception as e:
    print(e)

Cannot shift "2023-10-28 02:15:00+02:00" by "1D" (Would end up on time 2023-10-29 02:15:00 which is ambiguous on time zone Europe/Rome)


This is because 2:15 AM on October 28, 2023 actually represent two points in time on time zone `Europe/Rome`: before and after the DST kicking in. Exacly as for 1:15 AM on the 5t of November, 2023, on time zone `America/New_York`:

In [32]:
try:
    Time(2023,11,5,1,15,0, tz='America/New_York')
except Exception as e:
    print(e)

Sorry, time 2023-11-05 01:15:00 is ambiguous on time zone America/New_York without an offset. Use guessing=True to allow creating it with a guess.


You can force creating such time enabling the "guessing" mode, but it will only be possible to create one of the two:

In [33]:
Time(2023,11,5,1,15,0, tz='America/New_York', guessing=True)

Time 2023-11-05 01:15:00 is ambiguous on time zone America/New_York, assuming -05:00 UTC offset


Time: 1699164900.0 (2023-11-05 01:15:00 America/New_York)

To get the other one, you need to explicitly set the offset:

In [34]:
assert Time(2023,11,5,1,15,0, offset=-3600*4, tz='America/New_York') == Time.from_iso('2023-11-05T01:15:00-04:00', tz='America/New_York')

...or just add the necessary hours to a previous point in time, which is an operation always allowed:

In [35]:
Time(2023,11,5,0,15,0, tz='America/New_York') + TimeSpan('1h')

Time: 1699161300.0 (2023-11-05 01:15:00 America/New_York DST)

In [36]:
Time(2023,11,5,0,15,0, tz='America/New_York') + TimeSpan('2h')

Time: 1699164900.0 (2023-11-05 01:15:00 America/New_York)

### Where to go from here

You can check out the [API documentation](https://propertime.readthedocs.io), or you can just `pip install propertime` and give it a try!